# Day 4
## Part 1

In [2]:
let parseNumbers (numbers : string) =
    numbers.Split(' ', StringSplitOptions.RemoveEmptyEntries)
    |> Array.map int

type Card = { Number : int; Winners : Set<int>; Randoms : Set<int> }

let parseCard (line : string) =
    let sections = line.Split [| ':'; '|' |]
    let number = sections[0].Split([| ' ' |], StringSplitOptions.RemoveEmptyEntries) |> Array.item 1 |> int
    let winners = parseNumbers sections[1] |> Set.ofArray
    let randoms = parseNumbers sections[2] |> Set.ofArray
    { Number = number; Winners = winners; Randoms = randoms }

let scoreCard (card : Card) =
    let winningNumbers = Set.intersect card.Winners card.Randoms
    match winningNumbers.Count with 
    | 0 -> 0
    | n -> 1 <<< (n-1)


In [3]:
#r "nuget: FsUnit.xUnit"
open FsUnitTyped

let testData = 
    [
        "Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53"
        "Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19"
        "Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1"
        "Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83"
        "Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36"
        "Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11"
    ]   

let testScore =
    testData 
    |> List.map parseCard 
    |> List.sumBy scoreCard   

testScore |> shouldEqual 13  

Installed Packages FsUnit.xUnit, 5.6.1

In [4]:
open System.IO
let input = File.ReadAllLines "input_04.txt"

let result1 =
    input
    |> Array.map parseCard 
    |> Array.sumBy scoreCard   


In [5]:
printfn "Result 1: %d" result1
result1 |> shouldEqual 20829 // In case I break it

Result 1: 20829


## Part 2

In [13]:
let numberedCards input =
    input
    |> Seq.mapi (fun i c -> (i+1), (parseCard c))
    |> Map.ofSeq

let winCards (card : Card) =
    Set.intersect card.Winners card.Randoms
    |> Set.count

let processCards input =
    let numberedCards = numberedCards input

    let rec processCard (cardsToDo : Card list) (cardCounts : Map<int, int>) =
        match cardsToDo with
        | [] -> cardCounts
        | (card::rest) ->
            let copies = cardCounts[card.Number]
            
            let cardCounts = 
                List.init (winCards card) (fun i -> card.Number + 1 + i)
                |> List.fold (
                    fun counts n -> 
                        let c = counts |> Map.find n
                        counts |> Map.add n (c+copies)
                    ) cardCounts
            processCard rest cardCounts

    let cardCounts = numberedCards.Values |> Seq.map (fun c -> c.Number, 1) |> Map.ofSeq
    processCard (List.ofSeq numberedCards.Values) cardCounts
    |> Map.values
    |> Seq.sum
    

In [14]:
let test2 = processCards testData
test2 |> shouldEqual 30

In [16]:
let result2 = processCards input
printfn "Result 2: %d" result2
result2 |> shouldEqual 12648035 // In case I break it

Result 2: 12648035
